In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data/input/combined.csv')
df.info()

concepts = pd.read_csv('../data/input/concepts.csv')
concepts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76893 entries, 0 to 76892
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   parameter_name  76893 non-null  object 
 1   concept_label   76893 non-null  object 
 2   hospital_name   76893 non-null  object 
 3   ehr_name        76893 non-null  object 
 4   table           76893 non-null  object 
 5   parameter_id    55008 non-null  object 
 6   atc             8840 non-null   object 
 7   unit            76893 non-null  object 
 8   num_units       75621 non-null  float64
 9   amin            41851 non-null  float64
 10  amax            41851 non-null  float64
 11  p25             41851 non-null  float64
 12  p50             41851 non-null  float64
 13  p75             41851 non-null  float64
 14  top_n           76893 non-null  object 
 15  num_records     76892 non-null  float64
 16  num_patients    75621 non-null  float64
 17  perc_patients   75502 non-null 

In [3]:
# transform hospital names to ids
HOSP_TO_ID_DICT = {k: i for k,i in zip(sorted(df.hospital_name.unique()), range(df.hospital_name.nunique()))}
ID_TO_HOSP_DICT = {i: k for k,i in HOSP_TO_ID_DICT.items()}

df["hospital_name"] = df.hospital_name.map(HOSP_TO_ID_DICT)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76893 entries, 0 to 76892
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   parameter_name  76893 non-null  object 
 1   concept_label   76893 non-null  object 
 2   hospital_name   76893 non-null  int64  
 3   ehr_name        76893 non-null  object 
 4   table           76893 non-null  object 
 5   parameter_id    55008 non-null  object 
 6   atc             8840 non-null   object 
 7   unit            76893 non-null  object 
 8   num_units       75621 non-null  float64
 9   amin            41851 non-null  float64
 10  amax            41851 non-null  float64
 11  p25             41851 non-null  float64
 12  p50             41851 non-null  float64
 13  p75             41851 non-null  float64
 14  top_n           76893 non-null  object 
 15  num_records     76892 non-null  float64
 16  num_patients    75621 non-null  float64
 17  perc_patients   75502 non-null 

In [4]:
df.hospital_name.nunique()

30

In [5]:
df.groupby('ehr_name').hospital_name.nunique()

ehr_name
EPIC     6
HIX     13
MV      11
Name: hospital_name, dtype: int64

In [6]:
df.groupby(['ehr_name', 'hospital_name'])['num_patients'].max()

ehr_name  hospital_name
EPIC      1                222.0
          8                141.0
          20                82.0
          23                55.0
          24               158.0
          27               184.0
HIX       4                 21.0
          5                249.0
          6                123.0
          7                408.0
          9                109.0
          12                68.0
          14                43.0
          16               113.0
          18                54.0
          21                57.0
          22                27.0
          26                82.0
          28               338.0
MV        0                134.0
          2                105.0
          3                 10.0
          10                54.0
          11               175.0
          13                73.0
          15                81.0
          17                52.0
          19               244.0
          25               286.0
          29       

In [7]:
df.loc[df['concept_label'].str.contains('fio2_set')].groupby(['ehr_name', 'hospital_name'])['parameter_name'].agg([list, 'count'])

list  \
ehr_name hospital_name                                                      
EPIC     1              [Fi02;R UMCA AN FIO2, FiO2 (%);R UMCA ICU FIO2...   
         8              [Zuurstof (beademd);R FIO2 (BEADEMD), Ingestel...   
         20             [FiO2 [%];R UMCN FIO2, FiO2 (ingesteld) [%];R ...   
         23             [FIO2 (%)(oud);SZ_IP IC R FIO2 INSTELLING, FiO...   
         24             [FiO2 (%);R FIO2, FiO2 bead;SAZ KLIN R FIO2 BE...   
         27             [Fi02;R UMCA AN FIO2, FiO2 (%);R UMCA ICU FIO2...   
HIX      4                                     [Maxiflow set, FiO2 (set)]   
         5                                                   [FiO2 (set)]   
         6                                                 [Maxiflow set]   
         7                                     [Maxiflow set, FiO2 (set)]   
         9                                                   [FiO2 (set)]   
         14                          [Maxiflow set, FiO2 (set), FiO2 (%)]   
         16                                                   [FiO2(set)]   
         18                                                  [FiO2 (set)]   
         21                                                  [FiO2 (set)]   
         22             [Maxiflow set, FiO2 (set), Saturatie; 40% FiO2...   
         26                                                   [FiO2(set)]   
         28                                                  [FiO2 (set)]   
MV       0                                  [FiO2 Primus, FiO2 beademing]   
         2                                                     [(I) FiO2]   
         3                                  [FiO2 sett., FiO2 Inst Servo]   
         10                                                  [FiO2 (Set)]   
         11                                                  [FiO2 - set]   
         13                                             [V500_FiO2 (set)]   
         15                                    [FiO2 (Set) U, FiO2 (Set)]   
         19             [Zuurstoftherapie - Zuurstof percentage, O2 Fr...   
         25             [Ventilator Instellingen Handmatig FiO2, Elisa...   
         29                 [Fractie ingeademde zuurstof, Ingesteld FiO2]   

                        count  
ehr_name hospital_name         
EPIC     1                  3  
         8                  2  
         20                 2  
         23                 4  
         24                 4  
         27                 3  
HIX      4                  2  
         5                  1  
         6                  1  
         7                  2  
         9                  1  
         14                 3  
         16                 1  
         18                 1  
         21                 1  
         22               378  
         26                 1  
         28                 1  
MV       0                  2  
         2                  1  
         3                  2  
         10                 1  
         11                 1  
         13                 1  
         15                 2  
         19                 2  
         25                 2  
         29                 2

In [8]:
df.loc[df['concept_label'] == 'unmapped']['perc_patients'].describe()

count    36746.000000
mean         0.163665
std          0.266779
min          0.001252
25%          0.011264
50%          0.035044
75%          0.168675
max          1.000000
Name: perc_patients, dtype: float64

In [9]:
df.loc[df['concept_label'] == 'unmapped']['table'].value_counts()

table
measurements            28515
orders                   4734
range_signals_joined     2243
events                   1785
lda                       590
medications               101
fluid_in                   82
fluid_out                  19
Name: count, dtype: int64

In [10]:
df.loc[(df['concept_label'] == 'unmapped') & (df['table'] == 'medications')]

,parameter_name,concept_label,hospital_name,ehr_name,table,parameter_id,atc,unit,num_units,amin,amax,p25,p50,p75,top_n,num_records,num_patients,perc_patients,type
4225,AEROCHAMBER FLOW-VU MET MONDSTUK BLAUW VOLW AS,unmapped,1,EPIC,medications,NaN,NaN,['stuk'],1.0,1.0,1.0,1.0,1.0,1.0,['1'],1.0,1.0,0.004505,bool
4372,COVID-19 PBM PAKKET KORTE TERMIJN,unmapped,1,EPIC,medications,NaN,NaN,['stuk'],1.0,NaN,NaN,NaN,NaN,NaN,['3; 4 stuk'],1.0,1.0,0.004505,bool
4415,DRUKSYSTEEM NATRIUMCHLORIDE,unmapped,1,EPIC,medications,NaN,NaN,['ml/uur'],1.0,NaN,NaN,NaN,NaN,NaN,"['5-15', '0-15', '5-10']",250.0,216.0,0.972973,cat
4703,NIET-ASSORTIMENTSARTIKEL,unmapped,1,EPIC,medications,NaN,NaN,"['Puf', 'stuk', 'mcg', 'mg', 'druppel', 'table...",9.0,1.0,1500.0,1.0,5.0,200.0,"['1', '250', '1500']",17.0,14.0,0.063063,num
4721,"NOVOFINE NAALD 0,25X 6MM 32G",unmapped,1,EPIC,medications,NaN,NaN,['Eenheden'],1.0,NaN,NaN,NaN,NaN,NaN,['40; 5 stuk'],1.0,1.0,0.004505,bool
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71496,STUDIEMEDICATIE (NIET IN EPIC VOORGESCHREVEN),unmapped,27,EPIC,medications,NaN,NaN,['mg'],1.0,400.0,800.0,500.0,600.0,700.0,"['400', '800']",2.0,1.0,0.005435,num
71553,VITORTHO MAGNESIUM BISGLYCINAAT 100MG+TAURINE ...,unmapped,27,EPIC,medications,NaN,NaN,['tablet'],1.0,1.0,1.0,1.0,1.0,1.0,['1'],1.0,1.0,0.005435,bool
71561,WATER STERIEL VOOR INHALATIE,unmapped,27,EPIC,medications,NaN,NaN,['ml'],1.0,10.0,10.0,10.0,10.0,10.0,['10'],2.0,2.0,0.010870,bool
71568,ZZ IMS TEMPLATE,unmapped,27,EPIC,medications,NaN,NaN,"['mg', 'mcg']",2.0,5.0,800.0,95.0,150.0,150.0,"['150', '200', '30']",51.0,26.0,0.141304,num


In [11]:
df.loc[df['concept_label'].str.contains('respiratory_rate_set')].groupby(['ehr_name', 'hospital_name'])['parameter_name'].agg([list, 'count'])

list  \
ehr_name hospital_name                                                      
EPIC     1              [Ademfrequentie ;R UMCA ICU VENTILATOR RESP RA...   
         8              [Ademfrequentie;R VENT RESP RATE (SET), SIMV f...   
         20             [Beademingsfrequentie CMV;R UMCN BEADEMINGSFRE...   
         23             [Freq / RR;SZ_IP IC R FREQ / RR INSTELLING, Fr...   
         24             [Ademfrequentie (Set);SAZ KLIN R ADEMFREQUENTI...   
         27             [Ademfrequentie ;R UMCA ICU VENTILATOR RESP RA...   
HIX      4                                       [Respiration Rate (set)]   
         5                                         [Ademfreqentie (inst)]   
         6                                         [Ademfreqentie (inst)]   
         7              [Respiration Rate (set), Ademfreqentie (inst),...   
         9                         [Ademfreqentie (inst), Resp manditory]   
         14                        [Ademfreqentie (inst), Resp manditory]   
         18                        [Ademfreqentie (inst), Resp manditory]   
         21                                        [Ademfreqentie (inst)]   
         22                    [Back-up frequentie, Ademfreqentie (inst)]   
         26                                         [AdemContr, RespVent]   
         28                                      [Respiration Rate (set)]   
MV       0                                                   [Frequentie]   
         2                    [(I) Ademfreq SIMV set, (I) Ademfrequentie]   
         3              [Freq Resp. sett., Frequentie afspraak(beademi...   
         10             [Resp Rate instelling, SIMV frequentie, Ademha...   
         11                                        [Ademfrequentie - set]   
         13                                  [V500_Respiratory Rate(set)]   
         15             [Frequentie instelling servo (U), SIMV frequen...   
         17                                  [Ademfrequentie (Mandatory)]   
         19             [Ademfrequentie - Beademing - Set, Synchronize...   
         25             [Ventilator Instellingen Handmatig Frequentie,...   
         29                                       [Ingestelde frequentie]   

                        count  
ehr_name hospital_name         
EPIC     1                  1  
         8                  2  
         20                 2  
         23                 2  
         24                 2  
         27                 1  
HIX      4                  1  
         5                  1  
         6                  1  
         7                  3  
         9                  2  
         14                 2  
         18                 2  
         21                 1  
         22                 2  
         26                 2  
         28                 1  
MV       0                  1  
         2                  2  
         3                  4  
         10                 3  
         11                 1  
         13                 1  
         15                 5  
         17                 1  
         19                 2  
         25                 3  
         29                 1

In [12]:
r_parameters = df.parameter_name.shape[0]
r_parameters_unique = df.parameter_name.nunique()
r_parameters_unique_proportion = r_parameters_unique / r_parameters
r_mapped = (df.concept_label != 'unmapped').sum()
r_mapped_proportion = r_mapped / r_parameters
r_mapped_categories = df.concept_label.map(concepts.set_index('concept_label')['category'])
r_duplicates = df.loc[df.duplicated(subset=['parameter_name', 'hospital_name', 'table', 'parameter_id'], keep=False)].shape[0]
r_mapped_number = df.loc[df.concept_label != 'unmapped']['concept_label'].nunique()
r_mapped_non_med = df.loc[(df.concept_label != 'unmapped') & (~df.concept_label.str.startswith('med_'))]['concept_label'].nunique()

In [13]:
r_groups = concepts.loc[concepts['category'] != 'medication'].groupby('category').count().index.nunique()
r_labels = concepts.loc[concepts['category'] != 'medication'].groupby('category').count().sum()['concept_label']
r_labels_2 = concepts.loc[concepts['category'] != 'medication'].groupby('category', dropna=False).count().sum()['concept_label']
r_medication = concepts.groupby('category').count()['concept_label']['medication']

In [14]:
print(f"""
The parameter file contains {r_parameters} parameters, of which {r_parameters_unique} are unique ({r_parameters_unique_proportion:.2%}). Based on parameter name, hospital name, table of origin and parameter id, {r_duplicates} records are marked as full duplicates.

 Of all parameters, {r_mapped} are mapped to a concept label. This is {r_mapped_proportion:.2%} of the total parameters. The mapped parameters are distributed as follows:

{r_mapped_categories.value_counts(normalize=True).head(10).apply(lambda x: f'{x:.2%}')}

In total, {r_mapped_number} unique concepts were used to map the parameters. Of these, {r_mapped_non_med} are not medication concepts and {r_mapped_number - r_mapped_non_med} medication records.

There are {r_groups} groups of concepts and {r_labels} ({r_labels_2}) labels in the concepts file. With an additional {r_medication} medication concepts in the combined file.
""")


The parameter file contains 76893 parameters, of which 58848 are unique (76.53%). Based on parameter name, hospital name, table of origin and parameter id, 0 records are marked as full duplicates.

 Of all parameters, 38824 are mapped to a concept label. This is 50.49% of the total parameters. The mapped parameters are distributed as follows:

concept_label
unmapped                     49.51%
medication                   15.28%
laboratory value             11.88%
respiratory                   8.49%
fluid balance                 5.23%
hemodynamics                  4.49%
neurology                     1.02%
infectiology                  0.94%
clinical score                0.82%
renal replacement therapy     0.62%
Name: proportion, dtype: object

In total, 1679 unique concepts were used to map the parameters. Of these, 837 are not medication concepts and 842 medication records.

There are 15 groups of concepts and 982 (982) labels in the concepts file. With an additional 5456 medication c

## Prepare hospital data for combination with datawarehouse data

In [15]:
from pandas.api.types import is_numeric_dtype

DESCRIPTIVES_FILEPATHS = {
    "readable": "./hospital_descriptives/hospital_descriptives_readable.csv",
    "full": "./hospital_descriptives/hospital_descriptives.csv",
}

ehr_rename_dict = {
        "MV": "MetaVision",
        "mv": "MetaVision",
        "MetaVision": "MetaVision",
        "metavision": "MetaVision",
        "Metavision": "MetaVision",
        "hix": "HiX",
        "HIX": "HiX",
        "HiX": "HiX",
        "EPIC": "EPIC",
        "epic": "EPIC",
        "Epic": "EPIC",
    }

# Overwrite hospital names with ids
for k,i in DESCRIPTIVES_FILEPATHS.items():
    if k == "full":
        d = pd.read_csv(i, header=[0,1], skipinitialspace=True)
        unnamed_values = [sublist for item in d.columns for sublist in item if 'Unnamed' in sublist]
        rename_dict = {k: "" for k in unnamed_values}
        d = d.rename(columns=rename_dict)
    else:
        d = pd.read_csv(i)

    if not is_numeric_dtype(d["hospital"]):
        d["hospital"] = d["hospital"].map(HOSP_TO_ID_DICT)
    d["ehr"] = d["ehr"].map(ehr_rename_dict)
    assert d["hospital"].isna().sum() == 0, "Missing hospital name in dict!"
    d.to_csv(i, index=False)

desc = pd.read_csv("./hospital_descriptives/hospital_descriptives_readable.csv")

In [16]:
if "Patients (n)" not in desc.columns:
    desc["Patients (n)"] = pd.NA
    desc_columns = list(desc.columns[:3]) + ["Patients (n)"] + list(desc.columns[3:-1])
    desc = desc[desc_columns].copy()
if "" not in desc.columns:
    desc[""] = ""
    desc_columns = [""] + list(desc.columns[:-1])
    desc = desc[desc_columns].copy()
    desc.loc[desc["Mortality, hospital (%, n)"] == "-", ""] = "*"

In [17]:
df_to_add = df.groupby(["hospital_name", "ehr_name"])['num_patients'].max().reset_index()
df_to_add.columns = ['hospital', 'ehr', 'Patients (n)']
df_to_add['Hospital Type'] = "general"
df_to_add['Patients (n)'] = df_to_add['Patients (n)'].astype(int)
df_to_add['ehr'] = df_to_add['ehr'].map(ehr_rename_dict)
df_to_add[""] = "+"
hospital_ids_to_add = [x for x in df_to_add['hospital'].values if x not in desc['hospital'].values]
df_to_add = df_to_add.loc[df_to_add['hospital'].isin(hospital_ids_to_add)].astype(str).copy()

In [18]:
df_final = pd.concat([desc.astype(str), df_to_add], axis=0)
df_final['Hospital Type'] = df_final['Hospital Type'].map({"academic": "Academic", "peripheral": "General", "general": "General"})
df_final = df_final.rename(columns={"hospital": "Hospital ID", "ehr": "EHR"})
df_final = df_final.replace(np.nan, "-")
df_final = df_final.sort_values(['EHR', "", 'Hospital Type', 'Admissions (n)', 'Patients (n)'], ascending=[True, True, True, False, False]).copy()

TRAINING_IDS = [str(HOSP_TO_ID_DICT.get(hosp)) for hosp in ["amc", "vumc", "olvg", "erasmus"]]
df_final.loc[df_final["Hospital ID"].isin(TRAINING_IDS), ""] += "T"
df_final.loc[df_final[""].apply(len) > 1, ""] = df_final.loc[df_final[""].apply(len) > 1, ""].apply(lambda x: "/".join(list(x)))

df_final.to_csv("./hospital_descriptives/hospital_descriptives_readable_final.csv", index=False)

In [19]:
import joblib
joblib.dump(
    HOSP_TO_ID_DICT,
    "./HOSP_TO_ID_DICT.pkl"
)

['./HOSP_TO_ID_DICT.pkl']

In [20]:
df_final

,,Hospital ID,EHR,Hospital Type,Patients (n),Admissions (n),"Intubations (%, n)","Length of Stay, ICU (days, median (IQR))","Length of Stay, hospital (days, median (IQR))","Mortality, ICU (%, n)",...,"Chronic Dialysis (%, n)","Chronic Renal Insufficiency (%, n)","Diabetes (%, n)","Cardiovascular Insufficiency (%, n)","Respiratory Insufficiency (%, n)","Cirrhosis (%, n)","COPD (%, n)","Neoplasm (%, n)","Hematologic Malignancy (%, n)","Immunodeficiency (%, n)"
0,T,1,EPIC,Academic,196,210,80.8 (193),7.6 [3.5 - 12.9],15.3 [9.5 - 22.9],31.0 (210),...,1.1 (87),9.2 (87),18.4 (87),0.0 (87),1.1 (87),0.0 (87),16.1 (87),1.1 (87),3.4 (87),14.9 (87)
2,T,27,EPIC,Academic,161,173,80.9 (152),8.6 [3.5 - 17.4],16.6 [9.5 - 26.7],26.6 (173),...,0.0 (144),2.8 (144),25.0 (144),2.1 (144),3.5 (144),1.4 (144),10.4 (144),2.1 (144),3.5 (144),15.3 (144)
1,,20,EPIC,Academic,118,121,84.1 (107),14.4 [5.2 - 25.8],26.1 [15.1 - 42.0],18.2 (121),...,0.9 (117),5.9 (101),19.7 (117),6.0 (117),1.7 (117),0.0 (117),6.8 (117),1.7 (118),5.1 (117),18.8 (117)
4,,23,EPIC,General,44,49,87.8 (41),6.3 [0.6 - 10.4],19.3 [7.9 - 26.6],18.4 (49),...,-,-,-,-,-,-,-,-,-,-
5,,24,EPIC,General,249,266,41.6 (255),6.4 [3.1 - 14.8],15.8 [8.9 - 27.8],23.7 (266),...,0.0 (244),0.4 (244),23.8 (244),4.9 (244),6.6 (244),0.0 (244),7.4 (244),2.0 (244),2.9 (244),8.2 (244)
3,,8,EPIC,General,113,116,86.0 (114),3.0 [1.2 - 11.3],6.1 [2.5 - 16.8],13.8 (116),...,0.0 (113),4.4 (113),23.9 (113),1.8 (113),5.3 (113),0.0 (113),9.7 (113),0.0 (113),0.9 (113),3.5 (113)
6,T,7,HiX,Academic,128,131,85.8 (106),12.6 [6.0 - 23.9],18.3 [9.9 - 34.0],25.2 (131),...,0.0 (35),2.9 (35),20.0 (35),0.0 (35),0.0 (35),0.0 (35),8.6 (35),0.0 (35),2.9 (35),5.7 (35)
15,,26,HiX,General,80,88,88.5 (78),7.4 [3.0 - 14.1],11.0 [5.8 - 25.3],28.4 (88),...,1.4 (73),5.5 (73),30.1 (73),0.0 (73),9.6 (73),0.0 (73),20.5 (73),0.0 (73),1.4 (73),6.8 (73)
14,,21,HiX,General,55,59,79.2 (53),8.0 [1.1 - 18.9],16.6 [7.7 - 34.1],6.8 (59),...,0.0 (5),0.0 (5),40.0 (5),0.0 (5),0.0 (5),0.0 (5),20.0 (5),0.0 (5),0.0 (5),0.0 (5)
13,,18,HiX,General,53,53,78.8 (52),13.3 [4.1 - 22.5],23.4 [12.6 - 33.9],24.5 (53),...,0.0 (51),0.0 (51),15.7 (51),0.0 (51),0.0 (51),0.0 (51),3.9 (51),2.0 (51),2.0 (51),7.8 (51)
